# NOTEBOOK TO DEVELOP A CLOUD-INDEX FOR CORAL BLEACHING


In [1]:
from datetime import datetime
from netCDF4 import Dataset as ds
import numpy as np
import dateutil as du 
import parser
import pytz
import dateutil.parser 
import pandas as pd
from pandas import DataFrame
import csv
import glob, os
from os.path import expanduser
from pathlib import Path
import xarray as xr 

In [2]:
### Load .CSV and  
df = pd.read_csv('Export_Output.csv',encoding='latin-1', low_memory=False)
df = df[['pointid','lat','lon']]

In [3]:
len(df)

3670722

# EXTRACT THE CF FROM THE WARMEST MONTH OF THE YEAR ONLY IF DHW > 4

In [4]:
# Function to slice the filename with .nc extention, this will be used to open the files in directory
def file_info(ncfilename):
    with ds(ncfilename,'r') as ncdat:
        array = xr.open_dataset(ncfilename) # open_dataset works fine with either full path or filename.nc 
        return array  
def array_selDHW(array):
    dsloc = array.sel(lon=lon,lat=lat,method='nearest') 
    val_arr = dsloc['degree_heating_week']
    return val_arr
def array_selCF(array):
    dsloc = array.sel(lon=lon,lat=lat,method='nearest') 
    val_arr = dsloc['cloud_fraction']
    return val_arr

In [ ]:
home = Path(expanduser("~"))
y = []
for row in df.itertuples():
    lat = row.lat
    lon = row.lon
    path = home / Path ('C:/Users/bmped/Sync/INDEX/dhw_patmos.nc') 
    array = file_info(path)
    dhw = array_selDHW(array)
    cf = array_selCF(array)
    cf_occur = cf[np.where( dhw >= 4 )]
    y.append(cf_occur) # list of arrays.

In [ ]:
def INDEX(list_of_arrrays):
    values_ls = []
    lat_ls = []
    lon_ls = []
    for element in list_of_arrrays:
        if element.size == 0: 
           pass
        else:
            values = element.values.mean()
            lat = element.coords['lat'].values
            lon = element.coords['lon'].values
            values_ls.append(values)
            lat_ls.append(lat)
            lon_ls.append(lon)
    values_df = pd.DataFrame(values_ls,columns={'cf_a'})
    values_lat = pd.DataFrame(lat_ls, columns={'lat'})
    values_lon = pd.DataFrame(lon_ls, columns={'lon'})
    df = pd.concat([values_df,values_lat,values_lon], axis=1)
    mn = df['cf_a'].min()
    mx = df['cf_a'].max()
    diff = mx-mn
    df['cf_a_std'] = df['cf_a'].apply(lambda x: (x-mn)/diff)
    return df 

In [ ]:
idx = INDEX(y)

In [ ]:
idx.to_csv('map_sample.csv')
len(idx)